In [40]:
import numpy as np
import pandas as pd
from scipy.special import expit 
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "8" 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from scipy.special import softmax
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt



from scipy.special import expit
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    r2_score,
    roc_auc_score
)

In [41]:
df = pd.read_csv(r"MergedData.csv")
print("Shape:", df.shape)
df = df.drop(columns=["Unnamed: 0"])
df.head()

Shape: (11000, 285)


Rating  Binary Rating  Current Ratio  Long-term Debt / Capital  \
0       A              1         1.1507                    0.4551   
1  AAA-AA              1         1.1129                    0.0072   
2     BBB              1         1.9276                    0.2924   
3  AAA-AA              1         0.8358                    0.4708   
4       A              1         1.2931                    0.2644   

   Debt/Equity Ratio  Gross Margin  Operating Margin  EBIT Margin  \
0             0.8847       77.6230           19.4839      19.4839   
1             0.0073       43.6619           19.8327      19.8327   
2             0.4255       11.9008            3.3173       3.3173   
3             0.9491       64.5096           18.4549      18.4549   
4             0.4036        3.8385            1.3269       1.3269   

   EBITDA Margin  Pre-Tax Profit Margin  ...  Rating_Year_2014  \
0        28.9834                13.6093  ...                 0   
1        23.9379                20.8699  ...                 0   
2         3.6338                 3.0536  ...                 0   
3        27.9377                15.1135  ...                 0   
4         1.5847                 1.2304  ...                 0   

   Rating_Year_2015  Rating_Year_2016  Rating Agency_DBRS  \
0                 0                 0                   0   
1                 0                 0                   0   
2                 0                 0                   0   
3                 0                 0                   0   
4                 0                 0                   0   

   Rating Agency_Egan-Jones Ratings Company  Rating Agency_Fitch Ratings  \
0                                         0                            0   
1                                         0                            0   
2                                         0                            0   
3                                         0                            0   
4                                         0                            0   

   Rating Agency_HR Ratings de Mexico S.A. de C.V.  \
0                                                0   
1                                                0   
2                                                0   
3                                                0   
4                                                0   

   Rating Agency_Japan Credit Rating Agency,Ltd.  \
0                                              0   
1                                              0   
2                                              0   
3                                              0   
4                                              0   

   Rating Agency_Moody's Investors Service  \
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4                                        0   

   Rating Agency_Standard & Poor's Ratings Services  
0                                                 1  
1                                                 1  
2                                                 1  
3                                                 1  
4                                                 1  

[5 rows x 284 columns]

In [42]:
# --- Step 1: Prepare Data ---
X = df.drop(columns=["Rating", "Binary Rating"])
y_br = df["Binary Rating"].astype(int)
y_rating = df["Rating"]

# Encode multi-class target
le = LabelEncoder()
y_r = le.fit_transform(y_rating)
n_classes = len(le.classes_)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = np.hstack([np.ones((X_scaled.shape[0], 1)), X_scaled])


X_train, X_test, y_br_train, y_br_test, y_r_train, y_r_test = train_test_split(
    X_scaled, y_br, y_r, test_size=0.2, random_state=42
)

In [43]:
from sklearn.linear_model import LogisticRegression

# Binary classification
log_clf = LogisticRegression(max_iter=10)
log_clf.fit(X_train, y_br_train)

# Predictions
bin_pred_train = log_clf.predict(X_train)
bin_pred_test = log_clf.predict(X_test)


c:\Users\ayush\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_r_train)

# Predictions
rating_pred_train = lda_clf.predict(X_train)
rating_pred_test = lda_clf.predict(X_test)


In [45]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print("---- BINARY ----")
print("Accuracy:", accuracy_score(y_br_test, bin_pred_test))
print("Precision:", precision_score(y_br_test, bin_pred_test))
print("Recall:", recall_score(y_br_test, bin_pred_test))
print("F1 Score:", f1_score(y_br_test, bin_pred_test))

print("\n---- MULTICLASS ----")
print("Accuracy:", accuracy_score(y_r_test, rating_pred_test))
print("Precision (macro):", precision_score(y_r_test, rating_pred_test, average="macro"))
print("Recall (macro):", recall_score(y_r_test, rating_pred_test, average="macro"))
print("F1 Score (macro):", f1_score(y_r_test, rating_pred_test, average="macro"))


---- BINARY ----
Accuracy: 0.7809090909090909
Precision: 0.7955665024630542
Recall: 0.8959778085991679
F1 Score: 0.8427919112850619

---- MULTICLASS ----
Accuracy: 0.485
Precision (macro): 0.4684324182800738
Recall (macro): 0.4906332717953588
F1 Score (macro): 0.475279590136701


In [46]:
comparison = pd.DataFrame({
    "y_br": y_br_test,
    "bin_pred": bin_pred_test,
    "y_rating": le.inverse_transform(y_r_test),
    "rating_pred": le.inverse_transform(rating_pred_test)
})

comparison.head()

y_br  bin_pred     y_rating  rating_pred
107      0         0  B and Below  B and Below
5484     0         1           BB          BBB
6998     1         1          BBB  B and Below
3984     1         1            A          BBB
3111     0         0  B and Below           BB

## Original data

---- BINARY ----<br>
Accuracy: 0.8430493273542601<br>
Precision: 0.8589511754068716<br>
Recall: 0.9143407122232916 <br>
F1 Score: 0.8857808857808858<br>

---- MULTICLASS ----<br>
Accuracy: 0.5586162716207559<br>
Precision (macro): 0.5393707448809837<br>
Recall (macro): 0.5318763644011734<br>
F1 Score (macro): 0.5327518157048322<br>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>y_br</th>
      <th>bin_pred</th>
      <th>y_rating</th>
      <th>rating_pred</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>4756</th>
      <td>1</td>
      <td>1</td>
      <td>BBB</td>
      <td>A</td>
    </tr>
    <tr>
      <th>7379</th>
      <td>0</td>
      <td>0</td>
      <td>B and Below</td>
      <td>B and Below</td>
    </tr>
    <tr>
      <th>6093</th>
      <td>1</td>
      <td>1</td>
      <td>A</td>
      <td>A</td>
    </tr>
    <tr>
      <th>586</th>
      <td>0</td>
      <td>0</td>
      <td>B and Below</td>
      <td>B and Below</td>
    </tr>
    <tr>
      <th>4791</th>
      <td>1</td>
      <td>1</td>
      <td>BBB</td>
      <td>BBB</td>
    </tr>
  </tbody>
</table>
</div>

## Synthetic Data

---- BINARY ----<br>
Accuracy: 0.7786363636363637<br>
Precision: 0.7956656346749226<br>
Recall: 0.891123439667129<br>
F1 Score: 0.8406934903500164<br>

---- MULTICLASS ----<br>
Accuracy: 0.485<br>
Precision (macro): 0.4684324182800738<br>
Recall (macro): 0.4906332717953588<br>
F1 Score (macro): 0.475279590136701<br>


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>y_br</th>
      <th>bin_pred</th>
      <th>y_rating</th>
      <th>rating_pred</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>107</th>
      <td>0</td>
      <td>0</td>
      <td>B and Below</td>
      <td>B and Below</td>
    </tr>
    <tr>
      <th>5484</th>
      <td>0</td>
      <td>1</td>
      <td>BB</td>
      <td>BBB</td>
    </tr>
    <tr>
      <th>6998</th>
      <td>1</td>
      <td>1</td>
      <td>BBB</td>
      <td>B and Below</td>
    </tr>
    <tr>
      <th>3984</th>
      <td>1</td>
      <td>1</td>
      <td>A</td>
      <td>BBB</td>
    </tr>
    <tr>
      <th>3111</th>
      <td>0</td>
      <td>0</td>
      <td>B and Below</td>
      <td>BB</td>
    </tr>
  </tbody>
</table>
</div>